# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon.lexicon_memoris import lexicon_memoris, lithology_memoris, pollutant_memoris
from utils.io import boreholes_from_files, gdf_viewer
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 
import random
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/striplog/legend.py:687: UserWarning: This legend contains duplicate components.
  warnings.warn(w)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/striplog/legend.py:687: UserWarning: This legend contains duplicate components.
  warnings.warn(w)


## Reading of CSV files in a dir

In [2]:
#df = pd.read_csv('test_data_boreholes.csv', sep=',') # use english lexicon
df = pd.read_csv('test_litho_fr.csv', sep=',') # use french lexicon

## Data Checking

In [3]:
#pol = ['As', 'Ni', 'Cl', 'Hg', 'Pb', 'Zn']
pol = ['VR','VS','VI']
#df[['Pollutant', 'Concentration']] = ''
df[['Pollutant', 'Concentration']] = ''
df['Pollutant'] = df['Pollutant'].apply(lambda x: random.sample(pol, 1)[0])
df['Concentration'] = df['Concentration'].apply(lambda x: np.random.randint(0,100))
df['Unit'] = 'mg/l'

In [4]:
df['Lithology'] = df['Lithology'].replace({'Grès':'Marne', 'Calcaire':'Dolomie'})

In [5]:
df.loc[35.5, :] = ['BH007',152794.0, 121111.0, 60.0, 0.57, 45, 'Calcaire', 'Bleu', 4, 41, 45, 'Cl', 78, 'mg/l']
df.loc[25.5, :] = ['BH005',152473.0, 121460.0, 72.0, 0.61, 32, 'Schiste', 'gris', 3, 29, 32, 'Cl', 78, 'mg/l']
df.drop(index=4, inplace=True)
df.drop(columns='Colour', inplace=True)
df = df.sort_index().reset_index(drop=True)

In [6]:
gdf_viewer(df)

Rows : 996, columns : 13


interactive(children=(IntSlider(value=10, description='rows', max=996, min=10, readout=False), IntSlider(value…

## Create Boreholes from dataframes or files

In [7]:
bhs_df=[df[0:50]] # list of dataframes

In [8]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=False,
                                                litho_field=['Lithology', 'Pollutant'], 
                                                color_field='Colour', lexicon=lexicon_memoris, 
                                                diam_field='Diameter', thick_field='Thickness')

/home/yanathan/Projects/GSDMA/utils/io.py:469: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=bh_name))
/home/yanathan/Projects/GSDMA/utils/io.py:469: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Intervals.id to


Dataframe 0 processing...
|__ID:'BH001'
{'lithology': 'sable', 'pollutant': 'vr'}
{'lithology': 'argile', 'pollutant': 'vr'}
{'lithology': 'marne', 'pollutant': 'vs'}
{'lithology': 'charbon', 'pollutant': 'vi'}
Summary : [Striplog(4 Intervals, start=0.0, stop=28.0)]
|__ID:'BH002'
{'lithology': 'sable', 'pollutant': 'vi'}
{'lithology': 'argile', 'pollutant': 'vi'}
{'lithology': 'marne', 'pollutant': 'vi'}
{'lithology': 'charbon', 'pollutant': 'vr'}
{'lithology': 'dolomie', 'pollutant': 'vi'}
Summary : [Striplog(5 Intervals, start=0.0, stop=37.0)]
|__ID:'BH003'
{'lithology': 'sable', 'pollutant': 'vi'}
{'lithology': 'argile', 'pollutant': 'vs'}
{'lithology': 'marne', 'pollutant': 'vi'}
{'lithology': 'charbon', 'pollutant': 'vi'}
{'lithology': 'dolomie', 'pollutant': 'vs'}
Summary : [Striplog(5 Intervals, start=0.0, stop=18.0)]
|__ID:'BH004'
{'lithology': 'sable', 'pollutant': 'vr'}
{'lithology': 'argile', 'pollutant': 'vi'}
{'lithology': 'marne', 'pollutant': 'vi'}
{'lithology': 'charbo

In [9]:
boreholes

[<core.orm.BoreholeOrm>(Name=BH001, Length=28.0, Diameter=0.43, Intervals=4),
 <core.orm.BoreholeOrm>(Name=BH002, Length=37.0, Diameter=0.6, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH003, Length=18.0, Diameter=0.41, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH004, Length=26.0, Diameter=0.61, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH005, Length=32.0, Diameter=0.61, Intervals=6),
 <core.orm.BoreholeOrm>(Name=BH006, Length=26.0, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH007, Length=45.0, Diameter=0.57, Intervals=6),
 <core.orm.BoreholeOrm>(Name=BH008, Length=35.0, Diameter=0.39, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH009, Length=40.0, Diameter=0.5, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH10, Length=14.0, Diameter=0.31, Intervals=4)]

In [10]:
components

{0: Component({'lithology': 'marne', 'pollutant': 'vs'}),
 1: Component({'lithology': 'charbon', 'pollutant': 'vi'}),
 2: Component({'lithology': 'argile', 'pollutant': 'vr'}),
 3: Component({'lithology': 'sable', 'pollutant': 'vr'}),
 4: Component({'lithology': 'dolomie', 'pollutant': 'vi'}),
 5: Component({'lithology': 'marne', 'pollutant': 'vi'}),
 6: Component({'lithology': 'sable', 'pollutant': 'vi'}),
 7: Component({'lithology': 'argile', 'pollutant': 'vi'}),
 8: Component({'lithology': 'charbon', 'pollutant': 'vr'}),
 9: Component({'lithology': 'dolomie', 'pollutant': 'vs'}),
 10: Component({'lithology': 'argile', 'pollutant': 'vs'}),
 11: Component({'lithology': 'charbon', 'pollutant': 'vs'}),
 12: Component({'lithology': 'marne', 'pollutant': 'vr'}),
 13: Component({'lithology': 'schiste'}),
 14: Component({'lithology': 'dolomie', 'pollutant': 'vr'}),
 15: Component({'lithology': 'calcaire'}),
 16: Component({'lithology': 'sable', 'pollutant': 'vs'})}

In [11]:
len(boreholes), len(components)

(10, 17)

## Create a project and save data in a database

In [12]:
remove('tmp_files/tfe_orm_db.db')

In [13]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [14]:
Base.metadata.create_all(engine)

2021-07-02 21:33:45,120 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-02 21:33:45,121 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-07-02 21:33:45,122 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-02 21:33:45,124 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-07-02 21:33:45,125 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-02 21:33:45,126 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-07-02 21:33:45,126 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-02 21:33:45,128 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-07-02 21:33:45,128 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-02 21:33:45,130 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("IntervalData")
2021-07-02 21:33:45,130 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-02 21:33:45,131 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("IntervalData")
2021-07-02 21:33:45,132 INFO sqlalchemy.engine.E

In [15]:
Session = sessionmaker(bind=engine)
session = Session()

In [16]:
p = Project(session, legend_dict={'lithology':lithology_memoris, 'pollutant':pollutant_memoris}, 
            lexicon=lexicon_memoris, repr_attribute='lithology')

p.add_components(components)

2021-07-02 21:33:45,185 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-02 21:33:45,188 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-07-02 21:33:45,189 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ()
2021-07-02 21:33:45,195 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-07-02 21:33:45,196 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ((0, 'Marne, vs'), (1, 'Charbon, vi'), (2, 'Argile, vr'), (3, 'Sable, vr'), (4, 'Dolomie, vi'), (5, 'Marne, vi'), (6, 'Sable, vi'), (7, 'Argile, vi')  ... displaying 10 of 17 total bound parameter sets ...  (15, 'Calcaire'), (16, 'Sable, vs'))
2021-07-02 21:33:45,198 INFO sqlalchemy.engine.Engine COMMIT
2021-07-02 21:33:45,202 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-02 21:33:45,203 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Bor

Add boreholes into the project

In [17]:
for bh in boreholes:
    p.add_borehole(bh)

2021-07-02 21:33:45,213 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-07-02 21:33:45,214 INFO sqlalchemy.engine.Engine [generated in 0.00163s] ('BH001', 28.0, 0.43)
2021-07-02 21:33:45,217 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-07-02 21:33:45,218 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ((0, 101.0, 101.0, 101.0, 152600.0, 121039.0), (1, 97.0, 97.0, 97.0, 152600.0, 121039.0), (2, 97.0, 97.0, 97.0, 152600.0, 121039.0), (3, 93.0, 93.0, 93.0, 152600.0, 121039.0), (4, 93.0, 93.0, 93.0, 152600.0, 121039.0), (5, 81.0, 81.0, 81.0, 152600.0, 121039.0), (6, 81.0, 81.0, 81.0, 152600.0, 121039.0), (7, 73.0, 73.0, 73.0, 152600.0, 121039.0))
2021-07-02 21:33:45,221 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?)
2021-07-02 21:33:45,221 INFO sqlalchemy.e

TypeError: list indices must be integers or slices, not str

In [ ]:
p.add_link_between_components_and_intervals(link_intv_comp)

In [ ]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

In [ ]:
p.boreholes

In [ ]:
p.refresh(verbose=False)

In [ ]:
session.close()

# 2D/3D Display

In [ ]:
p.legend_dict.keys() #['pollutant']

In [ ]:
p.update_legend_cmap(update_legend=True)[1]

## Display one object (attribute = 'lithology')

In [ ]:
bh = 1
p.boreholes_3d[bh]._components

In [ ]:
p.boreholes_3d[bh].plot2d()

In [ ]:
p.boreholes_3d[bh].cmap

In [ ]:
sargs = dict(title='test', title_font_size=30, label_font_size=12, n_labels=0,
             fmt="%.2f", font_family="arial", color='k',interactive=True, 
             vertical=True, italic=True, shadow=False,)

In [ ]:
bh=1
p.boreholes_3d[bh].plot3d(show_legend=True, scalar_bar_args=sargs)

## Plot with another attribute

In [ ]:
p.update_legend_cmap(repr_attribute='pollutant')[1] #legend=lithology_memoris, update_legend=False)[0]

In [ ]:
p.boreholes_3d[bh].plot2d(repr_attribute='pollutant', repr_legend=pollutant_memoris)

In [ ]:
p.boreholes_3d[bh].plot3d(repr_attribute='pollutant', 
                          repr_legend=pollutant_memoris, 
                          show_legend=True)

### Display project boreholes

### Boreholes in 3D using X3D for interaction within the notebook